# Using transformers for DisjointTimeBasedCesnetDataset

### Import

In [1]:
import numpy as np
import logging

from cesnet_tszoo.utils.enums import AgreggationType, SourceType, TransformerType, DatasetType
from cesnet_tszoo.datasets import CESNET_TimeSeries24
from cesnet_tszoo.configs import DisjointTimeBasedConfig # Disjoint dataset MUST use DisjointTimeBasedConfig

from cesnet_tszoo.utils.transformer import Transformer # For creating custom Transformer

### Setting logger

In [2]:
logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s][%(name)s][%(levelname)s] - %(message)s")

### Preparing dataset

In [3]:
disjoint_dataset = CESNET_TimeSeries24.get_dataset(data_root="/some_directory/", source_type=SourceType.IP_ADDRESSES_SAMPLE, aggregation=AgreggationType.AGG_10_MINUTES, dataset_type=DatasetType.DISJOINT_TIME_BASED, display_details=True)

[2025-08-26 20:06:16,430][wrapper_dataset][INFO] - Dataset is disjoint_time_based. Use cesnet_tszoo.configs.DisjointTimeBasedConfig



Dataset details:

    AgreggationType.AGG_10_MINUTES
        Time indices: range(0, 40297)
        Datetime: (datetime.datetime(2023, 10, 9, 0, 3, 49, tzinfo=datetime.timezone.utc), datetime.datetime(2024, 7, 14, 21, 50, 52, tzinfo=datetime.timezone.utc))

    SourceType.IP_ADDRESSES_SAMPLE
        Time series indices: [ 11  20 101 103 118 ... 2003134 2008461 2011839 2022235 2044888], Length=1000; use 'get_available_ts_indices' for full list
        Features with default values: {'n_flows': 0, 'n_packets': 0, 'n_bytes': 0, 'n_dest_ip': 0, 'n_dest_asn': 0, 'n_dest_ports': 0, 'tcp_udp_ratio_packets': 0.5, 'tcp_udp_ratio_bytes': 0.5, 'dir_ratio_packets': 0.5, 'dir_ratio_bytes': 0.5, 'avg_duration': 0, 'avg_ttl': 0}
        
        Additional data: ['ids_relationship', 'weekends_and_holidays']
        


### Transformers

- Transformers are implemented as class.
    - You can create your own or use built-in one.
- Transformer is applied after `default_values` and fillers took care of missing values.
- One transformer is used for all time series.
- Transformer must implement `transform`.
- Transformer must implement `partial_fit` (unless transformer is already fitted and `partial_fit_initialized_transformers` is False).
- To use transformer, train set must be implemented (unless transformer is already fitted and `partial_fit_initialized_transformers` is False).
- You can change used transformer later with `update_dataset_config_and_initialize` or `apply_transformer`.

#### Built-in

In [4]:
# Options

## Supported
TransformerType.STANDARD_SCALER
TransformerType.L2_NORMALIZER
TransformerType.LOG_TRANSFORMER
TransformerType.MAX_ABS_SCALER
TransformerType.MIN_MAX_SCALER

<TransformerType.MIN_MAX_SCALER: 'min_max_scaler'>

In [5]:
config = DisjointTimeBasedConfig(train_ts=500, val_ts=None, test_ts=None, train_time_period=0.5, features_to_take=["n_flows", "n_packets"],
                           transform_with=TransformerType.MIN_MAX_SCALER, nan_threshold=0.5, random_state=1500)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 20:06:16,445][disjoint_time_based_config][INFO] - Quick validation succeeded.
[2025-08-26 20:06:16,486][disjoint_time_based_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 20:06:16,490][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 500/500 [00:00<00:00, 933.22it/s]
[2025-08-26 20:06:17,038][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_10_MINUTES
    Source: SourceType.IP_ADDRESSES_SAMPLE

    Time series
        Train time series IDs: [182151  10158  65072  10196 338309 ... 175742 659213  11188  73422 483796], Length=60
        Val time series IDs: None
        Test time series IDs: None
    Time periods
        Train time periods: range(0, 20149)
        Val time periods: None
        Test time periods: None
    Features
        Taken features: ['n_flows', 'n_packets']
        Default values: [0. 0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Sliding window
        Sliding window size: None
        Sliding window prediction size: None
        Sliding window step size: 1
    Fillers
        Filler type: None
    Transformers
        Transformer type: min_max_scaler
        Are transformers premade: False
        Are premade transformers partial_fitted:

In [6]:
disjoint_dataset.get_train_df(workers=0).head(10)

,id_ip,id_time,n_flows,n_packets
0,182151.0,0.0,0.000079,2.240487e-07
1,182151.0,1.0,0.000045,1.280278e-07
2,182151.0,2.0,0.000045,1.280278e-07
3,182151.0,3.0,0.000056,1.600348e-07
4,182151.0,4.0,0.000090,2.560557e-07
5,182151.0,5.0,0.000034,9.602087e-08
6,182151.0,6.0,0.000068,1.920417e-07
7,182151.0,7.0,0.000045,1.280278e-07
8,182151.0,8.0,0.000102,3.840835e-07
9,182151.0,9.0,0.000000,0.000000e+00


In [7]:
disjoint_dataset.get_transformers()

Or later with:

In [8]:
disjoint_dataset.update_dataset_config_and_initialize(transform_with=TransformerType.MIN_MAX_SCALER, partial_fit_initialized_transformers="config", workers=0)
# Or
disjoint_dataset.apply_transformer(transform_with=TransformerType.MIN_MAX_SCALER, partial_fit_initialized_transformers="config", workers=0)

[2025-08-26 20:06:17,277][cesnet_dataset][INFO] - Re-initialization is required.
[2025-08-26 20:06:17,313][disjoint_time_based_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 20:06:17,316][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 60/60 [00:00<00:00, 384.56it/s]
[2025-08-26 20:06:17,474][cesnet_dataset][INFO] - Config initialized successfully.
[2025-08-26 20:06:17,475][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-26 20:06:17,475][cesnet_dataset][INFO] - Re-initialization is required.
[2025-08-26 20:06:17,510][disjoint_time_based_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 20:06:17,513][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 60/60 [00:00<00:00, 402.17it/s]
[2025-08-26 20:06:17,665][cesnet_dataset][INFO] - Config initialized successfully.
[2025-08-26 20:06:17,665][cesnet_dataset][INFO] - Configuration has been changed successfu

#### Custom

You can create your own custom transformer. It is recommended to derive from Transformer base class.

In [9]:
class CustomTransformer(Transformer):
    def __init__(self):
        super().__init__()
        
        self.max = None
        self.min = None
    
    def transform(self, data):
        return (data - self.min) / (self.max - self.min)
    
    def fit(self, data):
        self.partial_fit(data)
    
    def partial_fit(self, data):
        
        if self.max is None and self.min is None:
            self.max = np.max(data, axis=0)
            self.min = np.min(data, axis=0)
            return
        
        temp_max = np.max(data, axis=0)
        temp = np.vstack((self.max, temp_max)) 
        self.max = np.max(temp, axis=0)
        
        temp_min = np.min(data, axis=0)
        temp = np.vstack((self.min, temp_min)) 
        self.min = np.min(temp, axis=0)            

In [10]:
config = DisjointTimeBasedConfig(train_ts=500, val_ts=None, test_ts=None, train_time_period=0.5, features_to_take=["n_flows", "n_packets"],
                           transform_with=CustomTransformer, nan_threshold=0.5, random_state=1500)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 20:06:17,675][disjoint_time_based_config][INFO] - Quick validation succeeded.
[2025-08-26 20:06:17,710][disjoint_time_based_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 20:06:17,713][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 500/500 [00:00<00:00, 1067.98it/s]
[2025-08-26 20:06:18,185][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_10_MINUTES
    Source: SourceType.IP_ADDRESSES_SAMPLE

    Time series
        Train time series IDs: [182151  10158  65072  10196 338309 ... 175742 659213  11188  73422 483796], Length=60
        Val time series IDs: None
        Test time series IDs: None
    Time periods
        Train time periods: range(0, 20149)
        Val time periods: None
        Test time periods: None
    Features
        Taken features: ['n_flows', 'n_packets']
        Default values: [0. 0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Sliding window
        Sliding window size: None
        Sliding window prediction size: None
        Sliding window step size: 1
    Fillers
        Filler type: None
    Transformers
        Transformer type: CustomTransformer (Custom)
        Are transformers premade: False
        Are premade transformers par

In [11]:
disjoint_dataset.get_train_df(workers=0).head(10)

,id_ip,id_time,n_flows,n_packets
0,182151.0,0.0,0.000079,2.240487e-07
1,182151.0,1.0,0.000045,1.280278e-07
2,182151.0,2.0,0.000045,1.280278e-07
3,182151.0,3.0,0.000056,1.600348e-07
4,182151.0,4.0,0.000090,2.560557e-07
5,182151.0,5.0,0.000034,9.602087e-08
6,182151.0,6.0,0.000068,1.920417e-07
7,182151.0,7.0,0.000045,1.280278e-07
8,182151.0,8.0,0.000102,3.840835e-07
9,182151.0,9.0,0.000000,0.000000e+00


In [12]:
disjoint_dataset.get_transformers()

Or later with:

In [13]:
disjoint_dataset.update_dataset_config_and_initialize(transform_with=CustomTransformer, partial_fit_initialized_transformers="config", workers=0)
# Or
disjoint_dataset.apply_transformer(transform_with=CustomTransformer, partial_fit_initialized_transformers="config", workers=0)

[2025-08-26 20:06:18,376][cesnet_dataset][INFO] - Re-initialization is required.
[2025-08-26 20:06:18,410][disjoint_time_based_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 20:06:18,413][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 60/60 [00:00<00:00, 437.60it/s]
[2025-08-26 20:06:18,551][cesnet_dataset][INFO] - Config initialized successfully.
[2025-08-26 20:06:18,552][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-26 20:06:18,553][cesnet_dataset][INFO] - Re-initialization is required.
[2025-08-26 20:06:18,635][disjoint_time_based_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 20:06:18,638][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 60/60 [00:00<00:00, 452.09it/s]
[2025-08-26 20:06:18,773][cesnet_dataset][INFO] - Config initialized successfully.
[2025-08-26 20:06:18,774][cesnet_dataset][INFO] - Configuration has been changed successfu

#### Using already fitted transformer

- When `partial_fit_initialized_transformer` is False (default value), transformer has no requirement for `partial_fit` nor for train set.

In [14]:
config = DisjointTimeBasedConfig(train_ts=500, val_ts=None, test_ts=None, train_time_period=0.5, features_to_take=["n_flows", "n_packets"],
                           transform_with=CustomTransformer, nan_threshold=0.5, random_state=1500)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=False, workers=0)

fitted_transformer = disjoint_dataset.get_transformers()

[2025-08-26 20:06:18,777][disjoint_time_based_config][INFO] - Quick validation succeeded.
[2025-08-26 20:06:18,813][disjoint_time_based_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 20:06:18,816][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 500/500 [00:00<00:00, 1154.83it/s]
[2025-08-26 20:06:19,251][cesnet_dataset][INFO] - Config initialized successfully.


In [15]:
config = DisjointTimeBasedConfig(train_ts=500, val_ts=500, test_ts=None, train_time_period=0.5, val_time_period=0.5, features_to_take=["n_flows", "n_packets"],
                           transform_with=fitted_transformer, nan_threshold=0.5, random_state=999)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 20:06:19,255][disjoint_time_based_config][INFO] - Quick validation succeeded.
[2025-08-26 20:06:19,319][disjoint_time_based_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 20:06:19,322][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 500/500 [00:00<00:00, 1105.30it/s]
[2025-08-26 20:06:19,779][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 500/500 [00:00<00:00, 967.62it/s]
[2025-08-26 20:06:20,297][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_10_MINUTES
    Source: SourceType.IP_ADDRESSES_SAMPLE

    Time series
        Train time series IDs: [  4380  35210 322201    190 307400 ...  29194 617662 677144 211973 612051], Length=65
        Val time series IDs: [ 450863  259860  182151     118 1604957 ... 363446 811185 338309 100610 612150], Length=75
        Test time series IDs: None
    Time periods
        Train time periods: range(0, 20149)
        Val time periods: range(20149, 40298)
        Test time periods: None
    Features
        Taken features: ['n_flows', 'n_packets']
        Default values: [0. 0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Sliding window
        Sliding window size: None
        Sliding window prediction size: None
        Sliding window step size: 1
    Fillers
        Filler type: None
    Transformers
        Transformer type: C

In [16]:
disjoint_dataset.get_val_df(workers=0).head(10)

,id_ip,id_time,n_flows,n_packets
0,450863.0,20149.0,0.000056,1.920417e-07
1,450863.0,20150.0,0.000000,0.000000e+00
2,450863.0,20151.0,0.000000,0.000000e+00
3,450863.0,20152.0,0.000034,1.280278e-07
4,450863.0,20153.0,0.000034,1.600348e-07
5,450863.0,20154.0,0.000045,3.200696e-07
6,450863.0,20155.0,0.000023,1.280278e-07
7,450863.0,20156.0,0.000045,1.280278e-07
8,450863.0,20157.0,0.000023,6.401392e-08
9,450863.0,20158.0,0.000000,0.000000e+00


Below you can see how transformer works even without train set.

In [17]:
config = DisjointTimeBasedConfig(train_ts=None, val_ts=500, test_ts=None, val_time_period=0.5, features_to_take=["n_flows", "n_packets"],
                           transform_with=fitted_transformer, nan_threshold=0.5, random_state=999)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 20:06:20,569][disjoint_time_based_config][INFO] - Quick validation succeeded.
[2025-08-26 20:06:20,605][disjoint_time_based_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 20:06:20,608][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 500/500 [00:00<00:00, 1177.85it/s]
[2025-08-26 20:06:21,035][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_10_MINUTES
    Source: SourceType.IP_ADDRESSES_SAMPLE

    Time series
        Train time series IDs: None
        Val time series IDs: [  4380  35210 322201    190 307400 ...  29194 617662 677144 211973 612051], Length=65
        Test time series IDs: None
    Time periods
        Train time periods: None
        Val time periods: range(0, 20149)
        Test time periods: None
    Features
        Taken features: ['n_flows', 'n_packets']
        Default values: [0. 0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Sliding window
        Sliding window size: None
        Sliding window prediction size: None
        Sliding window step size: 1
    Fillers
        Filler type: None
    Transformers
        Transformer type: CustomTransformer (Custom)
        Are transformers premade: True
        Are premade transformers part

In [18]:
disjoint_dataset.get_val_df(workers=0).head(10)

,id_ip,id_time,n_flows,n_packets
0,4380.0,0.0,0.001253,0.000005
1,4380.0,1.0,0.001162,0.000004
2,4380.0,2.0,0.000756,0.000003
3,4380.0,3.0,0.000959,0.000004
4,4380.0,4.0,0.001219,0.000005
5,4380.0,5.0,0.001275,0.000005
6,4380.0,6.0,0.001456,0.000006
7,4380.0,7.0,0.001196,0.000004
8,4380.0,8.0,0.001433,0.000006
9,4380.0,9.0,0.001072,0.000004


##### Partial fitting on train set

Makes already fitted transformer to be fitted on new train set too. Must implement `partial_fit`.

In [19]:
config = DisjointTimeBasedConfig(train_ts=500, val_ts=500, test_ts=None, train_time_period=0.5, val_time_period=0.5, features_to_take=["n_flows", "n_packets"],
                           transform_with=fitted_transformer, partial_fit_initialized_transformer=True, nan_threshold=0.5, random_state=999)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 20:06:21,249][disjoint_time_based_config][INFO] - Quick validation succeeded.
[2025-08-26 20:06:21,317][disjoint_time_based_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 20:06:21,320][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 500/500 [00:00<00:00, 1105.22it/s]
[2025-08-26 20:06:21,776][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 500/500 [00:00<00:00, 920.09it/s]
[2025-08-26 20:06:22,321][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_10_MINUTES
    Source: SourceType.IP_ADDRESSES_SAMPLE

    Time series
        Train time series IDs: [  4380  35210 322201    190 307400 ...  29194 617662 677144 211973 612051], Length=65
        Val time series IDs: [ 143746 1604957   10196  605036   67484 ...     20 321100 191587 612150    118], Length=75
        Test time series IDs: None
    Time periods
        Train time periods: range(0, 20149)
        Val time periods: range(20149, 40298)
        Test time periods: None
    Features
        Taken features: ['n_flows', 'n_packets']
        Default values: [0. 0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Sliding window
        Sliding window size: None
        Sliding window prediction size: None
        Sliding window step size: 1
    Fillers
        Filler type: None
    Transformers
        Transformer type: C

In [20]:
disjoint_dataset.get_val_df(workers=0).head(10)

,id_ip,id_time,n_flows,n_packets
0,143746.0,20149.0,0.000023,5.121113e-07
1,143746.0,20150.0,0.000000,0.000000e+00
2,143746.0,20151.0,0.000068,9.282018e-07
3,143746.0,20152.0,0.000000,0.000000e+00
4,143746.0,20153.0,0.000000,0.000000e+00
5,143746.0,20154.0,0.000023,2.240487e-07
6,143746.0,20155.0,0.000000,0.000000e+00
7,143746.0,20156.0,0.000000,0.000000e+00
8,143746.0,20157.0,0.000000,0.000000e+00
9,143746.0,20158.0,0.000000,0.000000e+00
